In [105]:
!echo "Stay online!"

Stay online!


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!nvcc --version
!pip install cupy-cuda100
!pip install dhash

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
     |████████████████████████████████| 335.2MB 52kB/s 
  Created wheel for dhash: filename=dhash-1.3-cp36-none-any.whl size=7274 sha256=e49cec2106835ee8a019fb443bd79f6e3bb834c1af48a5528283523a9d0e6ee6
  Stored in directory: /root/.cache/pip/wheels/a4/cb/88/d7cf61a1ac52042d59cc0c7a130c04b1541dfd439570c6ea41
Successfully built dhash


In [3]:
!git clone https://github.com/HyeongminLEE/pytorch-sepconv.git
%cd pytorch-sepconv

Cloning into 'pytorch-sepconv'...
remote: Enumerating objects: 211, done.
remote: Total 211 (delta 0), reused 0 (delta 0), pack-reused 211
Receiving objects: 100% (211/211), 14.54 MiB | 34.07 MiB/s, done.
Resolving deltas: 100% (64/64), done.
/content/pytorch-sepconv


In [4]:
# creaz dataset

!rm -rf sep_eps
!rm -rf ds
!mkdir ds

from google.colab.patches import cv2_imshow

import cv2
import dhash
from PIL import Image
import os
import shutil

def get_num_bits_different(hash1, hash2):
    return bin(hash1 ^ hash2).count('1')

vids = ['at_s01e01.mp4', 'at_s01e02.mp4', 'at_s01e03.mp4', 'at_s01e04.mp4', 'sao_e01.mkv', 'Akira.mkv']
#vids = ['at_s01e01.mp4', 'at_s01e02.mp4', 'at_s01e03.mp4', 'sao_e01.mkv', 'Akira.mkv']

# used to save ds
fld = 0

for vid_path in vids:

  cap = cv2.VideoCapture('/content/drive/My Drive/__progetto int sys/data/' + vid_path)
  vid_name = vid_path.split('.')[0]
  totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  if totalFrames > 60000:
    totalFrames = 60000
  if vid_name == 'at_s01e01' or vid_name == 'at_s01e02' or vid_name == 'at_s01e03' or vid_name == 'at_s01e04':
    totalFrames = 15000
  main_dir = 'sep_eps/' + vid_name
  if os.path.exists(main_dir):
      shutil.rmtree(main_dir)
  os.makedirs(main_dir)

  # 0 - 2: frame ripetuto => non lo salvo
  # 0 - 9 per Akira
  thresh_rip = 2
  if vid_name == 'Akira':
    thresh_rip = 9
    # skip 2400 frames
    for _ in range(2400):
      _, _ = cap.read()

  id_scena = 1
  ret, prec = cap.read()
  path = main_dir + '/' + str(id_scena).zfill(3)
  if not os.path.exists(path):
      os.mkdir(path)
  cv2.imwrite(path + '/0.jpg', prec)
  x = int(prec.shape[0] / 5)
  y = int(prec.shape[1] / 5)
  for i in range(1, int(totalFrames)):
      try:
          ret, frame = cap.read()
          prec_res = cv2.resize(prec, (y, x))
          frame_res = cv2.resize(frame, (y, x))
          h1 = dhash.dhash_int(Image.fromarray(prec_res), size=16)
          h2 = dhash.dhash_int(Image.fromarray(frame_res), size=16)

          bit_diff = get_num_bits_different(h1, h2)

          if bit_diff > thresh_rip:
              # threshold trovato sperimentalmente per sep shot = 213
              # usando 205, i falsi positivi aumentavano solo di 5
              # meglio qualche falso pos in piu' che un falso neg
              if bit_diff > 204:
                  id_scena += 1
              path = main_dir + '/' + str(id_scena).zfill(3)
              if not os.path.exists(path):
                  os.mkdir(path)
              frame = cv2.resize(frame, (640, 360))
              cv2.imwrite(path + '/' + str(i) + '.jpg', frame)

          prec = frame

      except:
          print('frame finale: ', i)
  print(vid_name)

  #########################################################

  ep_path = 'sep_eps/' + vid_name
  scenes = os.listdir(ep_path)
  scenes.sort()

  da_canc = []
  for s in scenes:
      folder = os.path.join(ep_path, s)
      frames_number = os.listdir(folder)
      if len(frames_number)<3:
          da_canc.append(folder)

  for f in da_canc:
      shutil.rmtree(f)

  #########################################################

  dirs = os.listdir('sep_eps')
  triplets = []
  for main_dir in dirs:
    lista_dir = os.listdir(os.path.join('sep_eps', main_dir))
    lista_dir.sort()
    for d in lista_dir:
      dir_path = os.path.join('sep_eps', main_dir, d)
      toAppend = os.listdir(dir_path)
      if len(toAppend)>2:
        files = []
        for f in toAppend:
          files.append(os.path.join(dir_path, f))
        files.sort()
        for i in range(len(files)-2):
          triplets.append([files[i], files[i+1], files[i+2]])
        
  print(len(triplets))

  #########################################################

  # save ds
  for t in triplets:
    save_dir = 'ds/train/' + str(fld).zfill(7)
    os.makedirs(save_dir)
    for i in range(3):
      shutil.copyfile(t[i], save_dir + '/frame' + str(i).zfill(1) + '.jpg')
    fld += 1
    
  print(ep_path)
  shutil.rmtree(ep_path)

at_s01e01
14481
sep_eps/at_s01e01
at_s01e02
14487
sep_eps/at_s01e02
at_s01e03
14502
sep_eps/at_s01e03
at_s01e04
14508
sep_eps/at_s01e04
sao_e01
51219
sep_eps/sao_e01
Akira
41534
sep_eps/Akira


In [5]:
!ls ds/train | wc -l

150731


In [6]:
!wget --timestamping http://content.sniklaus.com/sepconv/network-l1.pytorch
!wget --timestamping http://content.sniklaus.com/sepconv/network-lf.pytorch

import torch
from collections import OrderedDict

state_dict = torch.load('network-l1.pytorch')

new_state_dict = OrderedDict()
for k, v in state_dict.items():
  name = 'get_kernel.' + k
  new_state_dict[name] = v

torch.save(new_state_dict, 'l1-mod.pytorch')

--2020-02-20 22:14:10--  http://content.sniklaus.com/sepconv/network-l1.pytorch
Resolving content.sniklaus.com (content.sniklaus.com)... 51.15.58.24
Connecting to content.sniklaus.com (content.sniklaus.com)|51.15.58.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86713338 (83M) [application/octet-stream]
Saving to: ‘network-l1.pytorch’

network-l1.pytorch  100%[===================>]  82.70M  20.0MB/s    in 4.1s    

2020-02-20 22:14:14 (20.0 MB/s) - ‘network-l1.pytorch’ saved [86713338/86713338]

--2020-02-20 22:14:25--  http://content.sniklaus.com/sepconv/network-lf.pytorch
Resolving content.sniklaus.com (content.sniklaus.com)... 51.15.58.24
Connecting to content.sniklaus.com (content.sniklaus.com)|51.15.58.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86713362 (83M) [application/octet-stream]
Saving to: ‘network-lf.pytorch’

network-lf.pytorch  100%[===================>]  82.70M  19.4MB/s    in 4.3s    

2020-02-20 22:14:29 (1

In [0]:
'''
    if args.load_model is not None:
        checkpoint = torch.load(args.load_model)
        kernel_size = args.kernel
        model = SepConvNet(kernel_size=kernel_size)
        state_dict = torch.load(args.load_model)
        model.load_state_dict(checkpoint)
'''

'\n    if args.load_model is not None:\n        checkpoint = torch.load(args.load_model)\n        kernel_size = args.kernel\n        model = SepConvNet(kernel_size=kernel_size)\n        state_dict = torch.load(args.load_model)\n        model.load_state_dict(checkpoint)\n'

In [0]:
!mkdir '/content/drive/My Drive/__progetto int sys/ckps/SepC/prv360/'

In [0]:
# cambiare png in jpg in TorchDB.py
# --out_dir
!python -W ignore train.py --train ds/train/ --out_dir '/content/drive/My Drive/__progetto int sys/ckps/SepC/prv360/' --load_model l1-mod.pytorch

Beanbags:      30.1056852893009861 
Dimetrodon:    42.6861146274119108 
DogDance:      30.4676435774180021 
Grove2:        36.5046969832322077 
Grove3:        30.9518060979988441 
Hydrangea:     36.9697155456712920 
MiniCooper:    30.4474475647105436 
RubberWhale:   44.4014943962311364 
Urban2:        41.2306808231382504 
Urban3:        40.7639663587337253 
Venus:         35.3823209254403039 
Walking:       30.9936826696528129 
Average:       35.9087712382450022 
Train Epoch: [0/10]        Step: [0/37683]       train loss: 0.0000439877876488  
Train Epoch: [0/10]        Step: [100/37683]     train loss: 0.0023066371213645  
Train Epoch: [0/10]        Step: [200/37683]     train loss: 0.0005659704329446  
Train Epoch: [0/10]        Step: [300/37683]     train loss: 0.0017804556991905  
Train Epoch: [0/10]        Step: [400/37683]     train loss: 0.0052480241283774  
Train Epoch: [0/10]        Step: [500/37683]     train loss: 0.0003994127619080  
Train Epoch: [0/10]        Step: [600/37